In [1]:
import sys
sys.path.append("..")
from MPRA_predict.utils import *
from MPRA_predict.datasets import *

from torch.utils.data import DataLoader
from enformer_pytorch import Enformer, from_pretrained

import tensorflow as tf
import tensorflow_hub as hub
import gc

2024-12-19 10:58:12.071152: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-19 10:58:12.077903: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734577092.085754  423234 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734577092.088231  423234 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-19 10:58:12.096898: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
model = from_pretrained('../pretrained_models/enformer_weights', use_tf_gamma=True)

dataset = SeqLabelDataset(
    data_path='data/enformer_sequences_test_100.csv',
    data_type='seq',
    input_column='seq',
    crop=False,
    padding=False,
    N_fill_value=0)
test_data_loader = DataLoader(dataset, batch_size=4, shuffle=False, num_workers=4, pin_memory=True)
pred = get_pred(model, test_data_loader)

np.save(f'data/Enformer_pred_use_tf_gamma_true.npy', pred)

100%|██████████| 25/25 [00:10<00:00,  2.29it/s]


In [3]:
model = from_pretrained('../pretrained_models/enformer_weights', use_tf_gamma=False)

dataset = SeqLabelDataset(
    data_path='data/enformer_sequences_test_100.csv',
    data_type='seq',
    input_column='seq',
    crop=False,
    padding=False,
    N_fill_value=0)
test_data_loader = DataLoader(dataset, batch_size=4, shuffle=False, num_workers=4, pin_memory=True)
pred = get_pred(model, test_data_loader)

np.save(f'data/Enformer_pred_use_tf_gamma_false.npy', pred)

100%|██████████| 25/25 [00:10<00:00,  2.31it/s]


In [4]:
model_tf = hub.load("../pretrained_models/enformer_tfhub").model

df = pd.read_csv('data/enformer_sequences_test_100.csv')
seqs = np.array(df['seq'])
seqs = pad_seqs(seqs, 393216)
onehots = strs2onehots(seqs, N_fill_value=0)
onehots_tf =  tf.convert_to_tensor(onehots, dtype=tf.float32)

batch_size = 4
pred = []
for i in tqdm(range(0, len(onehots_tf), batch_size)):
    batch_inputs = onehots_tf[i:i+batch_size]
    batch_pred = model_tf.predict_on_batch(batch_inputs)
    pred.append(batch_pred['human'])

pred = tf.concat(pred, axis=0)
print(pred.shape)  # [100, 896, 5313]

np.save(f'data/Enformer_pred_tensorflow.npy', pred)

I0000 00:00:1734577197.594517  423234 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 18500 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:06:00.0, compute capability: 8.9
100%|██████████| 25/25 [00:28<00:00,  1.14s/it]


(100, 896, 5313)


In [5]:
pred_list = []
pred = np.load(f'data/Enformer_pred_use_tf_gamma_true.npy')
pred_list.append(pred)
pred = np.load(f'data/Enformer_pred_use_tf_gamma_false.npy')
pred_list.append(pred)
pred = np.load(f'data/Enformer_pred_tensorflow.npy')
pred_list.append(pred)
target = np.load('data/enformer_targets_test_100.npy')
pred_list.append(target)
for i in range(len(pred_list)):
    print(pred_list[i].shape)
pred_list = np.stack(pred_list)
# print(pred_list.shape)

print(np.corrcoef(pred_list.reshape(pred_list.shape[0], -1)))

(100, 896, 5313)
(100, 896, 5313)
(100, 896, 5313)
(100, 896, 5313)
[[1.         0.93410556 0.99999976 0.77285384]
 [0.93410556 1.         0.93409537 0.74083462]
 [0.99999976 0.93409537 1.         0.77286362]
 [0.77285384 0.74083462 0.77286362 1.        ]]


# 结论：由于tensorflow和pytorch的某个函数实现不同
# pytorch enformer只有在seq_len=196608bp且use_tf_gamma=True才能得到和tensorflow enformer一样的效果

# 如果seq_len != 196608,应该如何调整位置编码

In [2]:
for target_length in [2]:
    model = from_pretrained('../pretrained_models/enformer_weights', use_tf_gamma=True, target_length=2)

    dataset = SeqLabelDataset(
        data_path='data/enformer_sequences_test_100.csv',
        data_type='seq',
        input_column='seq',
        crop=True,
        cropped_length=256,
        padding=False,
        N_fill_value=0)
    test_data_loader = DataLoader(dataset, batch_size=4, shuffle=False, num_workers=4, pin_memory=True)
    pred = get_pred(model, test_data_loader)

    torch.cuda.empty_cache()
    np.save(f'data/Enformer_pred_crop_256_use_tf_gamma_true.npy', pred)

100%|██████████| 25/25 [00:00<00:00, 63.01it/s]


In [3]:
for target_length in [2]:
    model = from_pretrained('../pretrained_models/enformer_weights', use_tf_gamma=False, target_length=2)

    dataset = SeqLabelDataset(
        data_path='data/enformer_sequences_test_100.csv',
        data_type='seq',
        input_column='seq',
        crop=True,
        cropped_length=256,
        padding=False,
        N_fill_value=0)
    test_data_loader = DataLoader(dataset, batch_size=4, shuffle=False, num_workers=4, pin_memory=True)
    pred = get_pred(model, test_data_loader)

    torch.cuda.empty_cache()
    np.save(f'data/Enformer_pred_crop_256_use_tf_gamma_false.npy', pred)

100%|██████████| 25/25 [00:00<00:00, 87.45it/s]


In [6]:
pred_list = []
pred = np.load(f'data/Enformer_pred_crop_256_use_tf_gamma_true.npy')
pred_list.append(pred)
pred = np.load(f'data/Enformer_pred_crop_256_use_tf_gamma_false.npy')
pred_list.append(pred)
pred = np.load(f'data/Enformer_pred_use_tf_gamma_true.npy')[:, 447:449]
pred_list.append(pred)
pred = np.load(f'data/Enformer_pred_use_tf_gamma_false.npy')[:, 447:449]
pred_list.append(pred)
pred = np.load(f'data/Enformer_pred_tensorflow.npy')[:, 447:449]
pred_list.append(pred)
for i in range(len(pred_list)):
    print(pred_list[i].shape)
pred_list = np.stack(pred_list)
print(pred_list.shape)

print(np.corrcoef(pred_list.reshape(pred_list.shape[0], -1)))

(100, 2, 5313)
(100, 2, 5313)
(100, 2, 5313)
(100, 2, 5313)
(100, 2, 5313)
(5, 100, 2, 5313)
[[1.         0.96993794 0.53496298 0.51787961 0.53503864]
 [0.96993794 1.         0.52440155 0.51254982 0.52440759]
 [0.53496298 0.52440155 1.         0.95737919 0.99999959]
 [0.51787961 0.51254982 0.95737919 1.         0.95733426]
 [0.53503864 0.52440759 0.99999959 0.95733426 1.        ]]
